# Исследование надежности заемщиков.

# Введение
## Цели проекта
1. Определить влияние наличие детей, семейного положения, дохода на возврат крелита в срок 

### Шаг 1. Обзор данных

In [69]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

In [70]:
df.info() # общая информация 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [71]:
df.head(10) # первые 10 строк датафрейма 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Шаг 2.1 Заполнение пропусков

In [72]:
df.isna().sum()# подсчет пропусковb

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [73]:
df.describe() # заранее посмотрим на аномалии в данных 

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [74]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


In [75]:
df['days_employed'] = pd.to_numeric(df['days_employed'],errors='coerce').abs() # переводим в столбец в численные данные и методом abc убираем отрицательные значения
df['total_income'] = pd.to_numeric(df['total_income'],errors='coerce') # то же самое что и с 'days_employed' но в столбце нет отрицательных значений



def replace_nan_median(column_name): # функция для замены пропущенных значений на медиану 
    df[column_name] = df[column_name].fillna(df[column_name].median())


replace_nan_median('days_employed') # вызов функции
replace_nan_median('total_income') # вызов  функции 
display(df.isna().sum()) # проверяем заменили ли мы пропуски медианой 


children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Перевод столбцов в значение int и убираем отрицательные значения 
Функция replace_nan_median для замены значений на медиану 

Пропущенные значения имели формат nan 
пропуски состовляли 10 % 
Пропуски могли возникнуть по двум причинам:
1 - Эти данные просто не указали
2 - Ошибка при выгрузке
Заполнять пропуски медианой лучше т.к она при сильных отклонениях в данных в ту или иную сторону покажет правельное значение
UPD Так как я делал старую версию проекта сначала я уже знал об отрицатильных значениях и убрал их раньше 

### Шаг 2.2 Проверка данных на аномалии и исправления.

In [76]:
display(df.describe())
display(df.info())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.538908,60378.032733,43.293380,0.817236,0.972544,0.080883,1.651595e+05
std,1.381587,133257.558514,12.574584,0.548138,1.420324,0.272661,9.786607e+04
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.608174,33.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,1.955436e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


None

Остались только странные значение в children -1 

In [77]:
df['children'] = df['children'].abs() 

еще было замечено отрицательное значение в столбце children исправим это

Предпологаю что с значением children -1 произошла та же ошибка что и отрицательным стажем 

### Шаг 2.3. Изменение типов данных.

In [78]:
df['days_employed'] = df['days_employed'].astype('int') 
df['total_income'] = df['total_income'].astype('int')
df.head(10)


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


отбрасываем дробную часть от столбцов days_employed и total_income

In [79]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 11.3 MB


In [80]:
df['days_employed'] = df['days_employed'].astype('int32') 
df['total_income'] = df['total_income'].astype('int32')
df['children'] = df['children'].astype('int32')
df['education_id'] = df['education_id'].astype('int32')
df['family_status_id'] = df['family_status_id'].astype('int32')
df['debt'] = df['debt'].astype('int32')
df['income_type'] = df['income_type'].astype('category')


df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [81]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int32
days_employed       21525 non-null int32
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int32
family_status       21525 non-null object
family_status_id    21525 non-null int32
gender              21525 non-null object
income_type         21525 non-null category
debt                21525 non-null int32
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: category(1), int32(6), int64(1), object(4)
memory usage: 8.8 MB


### Шаг 2.4. Удаление дубликатов.

In [82]:
duplicated_df = df[df.duplicated()]
display(duplicated_df)
df.duplicated().sum()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
4182,1,2194,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,2194,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,2194,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
7808,0,2194,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
8583,0,2194,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование
9238,2,2194,34,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для сдачи
9528,0,2194,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
9627,0,2194,56,среднее,1,женат / замужем,0,F,пенсионер,0,145017,операции со своей недвижимостью
10462,0,2194,62,среднее,1,женат / замужем,0,F,пенсионер,0,145017,покупка коммерческой недвижимости


54

подсчет явынх дубликатов

In [83]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum() 

0

удаление дубликтов и обновление индексов
проверяем удаление дубликатов 

In [84]:
display(df['children'].sort_values(ascending = True).unique())

display(df['education'].sort_values(ascending = True).unique())

display(df['education_id'].sort_values(ascending = True).unique())

display(df['family_status'].sort_values(ascending = True).unique())

display(df['family_status_id'].sort_values(ascending = True).unique())

display(df['gender'].sort_values(ascending = True).unique())

display(df['income_type'].sort_values(ascending = True).unique())

display(df['debt'].sort_values(ascending = True).unique())

display(df['purpose'].sort_values(ascending = True).unique())



array([ 0,  1,  2,  3,  4,  5, 20])

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

array([0, 1, 2, 3, 4])

array(['Не женат / не замужем', 'в разводе', 'вдовец / вдова',
       'гражданский брак', 'женат / замужем'], dtype=object)

array([0, 1, 2, 3, 4])

array(['F', 'M', 'XNA'], dtype=object)

[безработный, в декрете, госслужащий, компаньон, пенсионер, предприниматель, сотрудник, студент]
Categories (8, object): [безработный, в декрете, госслужащий, компаньон, пенсионер, предприниматель, сотрудник, студент]

array([0, 1])

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

смотрим уникальные значения в датафрейме 

In [96]:
display(df['education'].value_counts())
display(df['family_status'].value_counts())
display(df['income_type'].value_counts())
display(df['purpose'].value_counts())


среднее                13705
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

женат / замужем          12344
гражданский брак          4163
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

сотрудник          11091
компаньон           5080
пенсионер           3837
госслужащий         1457
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

посмотрим на неявные дубликаты и видим разный регист в education и family_status

In [97]:
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
display(df['education'].value_counts())
display(df['family_status'].value_counts())


среднее                15188
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12344
гражданский брак          4163
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

приведем все к нижнему регистру и проверим результат 

Для поиска янвых дубликатов использовали drop_duplicates()
Для неявных sort_values(ascending = True).unique()
Дубликаты появились из-за человеческого фактора, разный регистр 

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [98]:
data1 = df[['education_id', 'education']]
columns1 = ['education_id', 'education']
new_education = pd.DataFrame(data= data1, columns = columns1) # создаем новый датафрйем new_education
new_education =new_education.drop_duplicates().reset_index(drop = True)
display(new_education.head(10)) 

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [99]:
data2 = df[['family_status_id','family_status']]
columns2 = ['family_status_id','family_status']
new_family = pd.DataFrame(data= data2, columns = columns2)
new_family =new_family.drop_duplicates().reset_index(drop = True)
display(new_family.head(10))


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


создаем новый датафрйем

In [100]:
df = df.drop(columns=['education','family_status'])
display(df.head(20)) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_group,family_status_id_group,debt_group
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,с детьми,в браке,не было долгов
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,с детьми,в браке,не было долгов
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,бездетные,в браке,не было долгов
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,с детьми,в браке,не было долгов
4,0,340266,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,бездетные,гражданский брак,не было долгов
5,0,926,27,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью,бездетные,гражданский брак,не было долгов
6,0,2879,43,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью,бездетные,в браке,не было долгов
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования,бездетные,в браке,не было долгов
8,2,6929,35,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы,с детьми,гражданский брак,не было долгов
9,0,2188,41,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью,бездетные,в браке,не было долгов


удаляем столбцы из исохдного датафрейма 

### Шаг 2.6. Категоризация дохода.

In [86]:
def total_income_fun (income):
    if income <= 30000:
        return 'E'
    if  income >=30001 and income <= 50000:
        return 'D'
    if income >= 50001 and income <= 200000:
        return 'C'
    if income >= 200001 and income <= 1000000:
        return 'B'
    if income >= 1000001:
        return 'A'
df['total_income_category'] = df['total_income'].apply(total_income_fun)        
display(df['total_income_category'].value_counts())  


C    16033
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64

функция для присваивания катеогрии по доходу в качестве аргумента берет весь столбец total_income

### Шаг 2.7. Категоризация целей кредита.

In [87]:
def purpose_category_fun (purpose):
    if 'жиль' in purpose or 'недвиж' in purpose:
        return 'операции с недвижимостью'
    if 'авто' in purpose:
        return 'операции с автомобилем'
    if 'образов' in purpose:
        return 'получение образования'
    if 'свадь' in purpose:
        return 'проведение свадьбы'
    return 'цель не найдена'
df['purpose_category'] = df['purpose'].apply(purpose_category_fun) # применяем функцию к всему столбцу 
display(df['purpose_category'].value_counts())


операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2335
Name: purpose_category, dtype: int64

функция для опеределения целей кредита

### Ответы на вопросы.

In [88]:
def children_group (children):
    if children == 0:
        return 'бездетные'
    if children <= 20:
        return 'с детьми'
    return 'ошибка'

df['children_group'] = df['children'].apply(children_group)
display(df['children_group'].value_counts())


бездетные    14107
с детьми      7364
Name: children_group, dtype: int64

функция для катеогрий по кол-ву детей 
почему не может быть 20 детей ? может их усыновили ? 
Отнесем их к катеогрии с детьми
 

In [89]:
def family_status_group (family_status): 
    if family_status == 0:
        return 'в браке'
    if family_status == 1:
        return 'гражданский брак'
    return 'одинокие'
df['family_status_id_group'] = df['family_status_id'].apply(family_status_group)
display(df['family_status_id_group'].value_counts())

в браке             12344
одинокие             4964
гражданский брак     4163
Name: family_status_id_group, dtype: int64

функция для катеогрий по типу проживания 

In [90]:
def debt_group (debt): 
    if debt == 0:
        return 'не было долгов'
    return 'долг был'
df['debt_group'] = df['debt'].apply(debt_group)
display(df['debt_group'].value_counts())

не было долгов    19730
долг был           1741
Name: debt_group, dtype: int64

функция для группировки по долгу

##### Вопрос 1:

In [91]:
data_children = df.pivot_table(index = ['children_group'], columns = 'debt_group' , values = 'debt' ,aggfunc='count')
display(data_children) 


debt_group,долг был,не было долгов
children_group,,
бездетные,1063,13044
с детьми,678,6686


сортировка по детям 

In [92]:
data_family = df.pivot_table(index = ['family_status_id_group'], columns = 'debt_group' , values = 'debt' ,aggfunc='count')
display(data_family) 

debt_group,долг был,не было долгов
family_status_id_group,,
в браке,931,11413
гражданский брак,388,3775
одинокие,422,4542


сортировка по статус брака 

In [93]:
data_income = df.pivot_table(index = ['total_income_category'], columns = 'debt_group' , values = 'debt' ,aggfunc='count')
display(data_income) 

debt_group,долг был,не было долгов
total_income_category,,
A,2,23
B,356,4685
C,1360,14673
D,21,329
E,2,20


сортировка по доходу

In [94]:
data_purpose = df.pivot_table(index = ['purpose_category'], columns = 'debt_group' , values = 'debt' ,aggfunc='count')
display(data_purpose) 

debt_group,долг был,не было долгов
purpose_category,,
операции с автомобилем,403,3905
операции с недвижимостью,782,10032
получение образования,370,3644
проведение свадьбы,186,2149


сортировка по цели получения 

Тут демонстрируем сводные таблицы по катеоргиям ответ на все вопросы ниже 

##### Вывод 1:

In [95]:
def final_df (data):
    def final_row(row):
        return row['долг был'] / (row['долг был']+row['не было долгов'])*100
    data['проценты с проблемами'] = data.apply(final_row,axis=1)
    return data
display(final_df(data_children))
display(final_df(data_family))
display(final_df(data_income))
display(final_df(data_purpose))


debt_group,долг был,не было долгов,проценты с проблемами
children_group,,,
бездетные,1063,13044,7.535266
с детьми,678,6686,9.206953


debt_group,долг был,не было долгов,проценты с проблемами
family_status_id_group,,,
в браке,931,11413,7.542126
гражданский брак,388,3775,9.320202
одинокие,422,4542,8.501209


debt_group,долг был,не было долгов,проценты с проблемами
total_income_category,,,
A,2,23,8.000000
B,356,4685,7.062091
C,1360,14673,8.482505
D,21,329,6.000000
E,2,20,9.090909


debt_group,долг был,не было долгов,проценты с проблемами
purpose_category,,,
операции с автомобилем,403,3905,9.354689
операции с недвижимостью,782,10032,7.231367
получение образования,370,3644,9.217738
проведение свадьбы,186,2149,7.965739


функция для поиска зависимостей делит всех у кого были проблемы с возвратом на всех кто брал кредит 

1- Зависимость наличие детей и возврата в кредита в срок : Люди без детей 7% были проблемы с возвратом, с наличием детей 9%
2 - Семейное положение и возврат в срок : в браке (официальном) 7.5% проблем
гражданский 9.3% проблем
Одинокие - 8.5%
Самый низкий риск у людей в браке 
3- Доход на возврат в срок самая безрисковая категория D - 6% , самый выссокий риск у катеории E - 9 % (но там довольно маленькая выборка )
4 - Цели кредита самый низкий риск у операции с недвижимостью 7.2%, самый выскойи у операции с автомобилем 9.3 % 

## Общий вывод:

Наличие детей минимально влияет на проблемы с возвратом кредита 
Семейное положение так же имеет минимальное влияние на проблемы с возвратом кредита в срок порядка 2%
Самые большие отклонения находьтся в доходе там разница может достигать 3% 


Произвели группировку и рассчеты  по признакам : 
1. Наличие детей 
2. Семейное положение 
3. Общий доход 
Вывод: 
1. Наличие детей существено не сказывается на возврат в срок. Разница между группами между группами 1.7% в пользу группы без детей
2. Семейное положение так же не играет существенной роли на возврат в срок. Разница в группах 1.8% в полбзу группы  'в браке'
3. По котегориям дохода уже существует более заметная разница там разница между самой выско рисковой и самой низкорисковой группой 3%
4. По группам цели кредита самое большое колебание в группах 2.1%